In [1]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.static import teams
from json import JSONDecodeError
import pandas as pd
import numpy as np
import random
import requests
import math
import time

In [34]:
def str_to_mins(inp):
    lst = inp.split(":")
    sec = int(lst[0])*60 + int(lst[1])
    return sec

# Step 1: Gettting Record of ALL Games

### We're going to use the teamgamelog endpoint to get a record of all the games since 1997.

In [98]:
from nba_api.stats.endpoints import teamgamelog
from ast import literal_eval
import pandas as pd
import time
import timeit

def get_gamelogs(season_start=1997, season_end = 2018, verbose = False, saveas = None):
    yr_tms_df = pd.read_csv("../common-datasets/all_teams_per_year.csv")
    lst = []
    time_taken = [10]
    start = timeit.default_timer()
    for season in range(season_start, season_end + 1):
        print(season)
        for team_name, team_id, team_abbrev in list(literal_eval(x) for x in yr_tms_df[f"{season}"] if not pd.isnull(x)):
            print(team_name)
            time.sleep(0.36)
            successful = False
            while not successful:
                try:
                    get_start = timeit.default_timer()
                    games = teamgamelog.TeamGameLog(team_id, season = season, timeout=round(2*sum(time_taken)/len(time_taken), 0))
                    get_end = timeit.default_timer()
                    time_taken.append(get_end-get_start)
                    successful = True
                except Exception as e:
                    if verbose:
                        print(f"Timeout error on {season} {team_name}, taking a 3 second break before resuming")
                    time.sleep(3)

            gamelog = games.get_data_frames()[0]
            gamelog["SEASON"] = season
            lst.append(gamelog)
            
    games_concat = pd.concat(lst, axis = 0, ignore_index = True)
    if saveas:
        games_concat.to_csv(f"{saveas}.csv",index=False)
    return games_concat

In [140]:
lst = []

In [143]:
yr_tms_df = pd.read_csv("../common-datasets/all_teams_per_year.csv")
saveas = "all_gamelogs"
season_start = 1997
season_end = 2018
verbose = True
time_taken = [10]
start = timeit.default_timer()
for season in range(season_start, season_end + 1):
    print(season)
    for team_name, team_id, team_abbrev in list(literal_eval(x) for x in yr_tms_df[f"{season}"] if not pd.isnull(x)):
        time.sleep(0.5)
        successful = False
        while not successful:
            try:
                get_start = timeit.default_timer()
                games = teamgamelog.TeamGameLog(team_id, season = season, timeout=round(2*sum(time_taken)/len(time_taken), 0))
                get_end = timeit.default_timer()
                time_taken.append(get_end-get_start)
                successful = True
            except Exception as e:
                if verbose:
                    print(f"Timeout error on {season} {team_name}, taking a 3 second break before resuming")
                time.sleep(3)

        gamelog = games.get_data_frames()[0]
        gamelog["SEASON"] = season
        gamelog["TEAM_ID"] = team_id
        gamelog = gamelog.rename(columns={'Game_ID': 'GAME_ID'})
        lst.append(gamelog)

games_concat = pd.concat(lst, axis = 0, ignore_index = True)
games_concat = games_concat.drop_duplicates()
if saveas:
    games_concat.to_csv(f"{saveas}.csv",index=False)

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [144]:
games_concat

,Team_ID,GAME_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,SEASON,TEAM_ID
0,1610612747,0029701187,"APR 19, 1998",LAL vs. UTA,W,NaN,NaN,NaN,240,38,...,31,43,21,6,9,12,26,102,1997,1610612747
1,1610612747,0029701173,"APR 17, 1998",LAL vs. DAL,W,NaN,NaN,NaN,240,47,...,35,48,34,6,9,19,14,124,1997,1610612747
2,1610612747,0029701145,"APR 14, 1998",LAL @ DAL,W,NaN,NaN,NaN,240,45,...,37,56,24,5,3,16,21,111,1997,1610612747
3,1610612747,0029701137,"APR 13, 1998",LAL @ SAS,W,NaN,NaN,NaN,240,37,...,43,48,21,7,5,11,22,99,1997,1610612747
4,1610612747,0029701127,"APR 11, 1998",LAL @ GSW,W,NaN,NaN,NaN,240,35,...,27,47,23,6,5,21,21,96,1997,1610612747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55961,1610612765,0021800074,"OCT 27, 2018",DET vs. BOS,L,4.0,1.0,0.8,240,33,...,41,47,16,4,3,11,24,89,2018,1610612765
55962,1610612765,0021800063,"OCT 25, 2018",DET vs. CLE,W,4.0,0.0,1.0,240,41,...,35,48,21,4,5,20,21,110,2018,1610612765
55963,1610612765,0021800049,"OCT 23, 2018",DET vs. PHI,W,3.0,0.0,1.0,265,51,...,34,46,22,4,4,8,24,133,2018,1610612765
55964,1610612765,0021800031,"OCT 20, 2018",DET @ CHI,W,2.0,0.0,1.0,240,41,...,31,49,21,4,4,9,23,118,2018,1610612765


In [3]:
from nba_api.stats.endpoints import boxscoretraditionalv2
from ast import literal_eval
import pandas as pd
import time
import timeit

def get_boxscores(game_ids, verbose = False, saveas = None):
    total_ids = len(game_ids)
    lst = []
    time_taken = [10]
    start = timeit.default_timer()
    for i in range(len(game_ids)):
        time.sleep(0.36)
        gid = game_ids[i]

        successful = False
        while not successful:
            try:
                get_start = timeit.default_timer()
                bx = boxscoretraditionalv2.BoxScoreTraditionalV2(gid, timeout=round(2*sum(time_taken)/len(time_taken), 0))
                get_end = timeit.default_timer()
                time_taken.append(get_end-get_start)
                successful = True
            except Exception as e:
                if verbose:
                    print(f"Timeout error on iteration {i}, taking a 3 second break before resuming")
                time.sleep(3)

        players = bx.get_data_frames()[0]
        lst.append(players)
        if i%100 == 1:
            stop = timeit.default_timer()
            if verbose:
                print(f"{i-1}/{total_ids} done: {round(i/total_ids*100,1)}% ... has taken {round((stop-start)/60,0)} min, total projected: {round((stop-start)/(i/total_ids)/60, 2)} min, remaining projected: {round(((stop-start)/(i/total_ids)-(stop-start))/60, 2)} min")
    boxscores_concat = pd.concat(lst, axis = 0, ignore_index = True)
    if saveas:
        boxscores_concat.to_csv(f"{saveas}.csv",index=False)
    return boxscores_concat


# Step 3: Gettting Player-Seasons for Starters

### Using the leaguedashplayerstats endpoint, as well as the leaguedashplayerbiostats endpoint, we will get player-seasons from 1997-2018

In [4]:
from nba_api.stats.endpoints import leaguedashplayerstats
import pandas as pd
import requests
import time

def get_player_seasons(season_start=1997, season_end = 2018, per_mode = "PerPossession", verbose = False, saveas = None):
    #per_mode can be "PerPossession", "PerGame", or "Totals"
    lst = []
    time_taken = [10]
    start = timeit.default_timer()
    for season in range(season_start, season_end + 1):
        time.sleep(0.36)

        successful = False
        while not successful:
            try:
                get_start = timeit.default_timer()
                games = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = per_mode, season = f"{season}-{str(season+1)[-2:]}", timeout=round(2*sum(time_taken)/len(time_taken), 0))
                get_end = timeit.default_timer()
                time_taken.append(get_end-get_start)
                successful = True
            except Exception as e:
                if verbose:
                    print(f"Timeout error on season {season}-{str(season+1)[-2:]}, taking a 3 second break before resuming")
                time.sleep(3)

        gamelog = games.get_data_frames()[0]
        gamelog["SEASON"] = season
        lst.append(gamelog)
    season_infos = pd.concat(lst, axis = 0, ignore_index = True)
    if saveas:
        season_infos.to_csv(f"{saveas}.csv",index=False)
    return season_infos

In [5]:
from nba_api.stats.endpoints import leaguedashplayerbiostats
import pandas as pd
import requests
import time

def get_player_seasons_advanced(season_start=1997, season_end = 2018, verbose = False, saveas = None):
    #per_mode can be "PerPossession", "PerGame", or "Totals"
    lst = []
    time_taken = [10]
    start = timeit.default_timer()
    for season in range(season_start, season_end + 1):
        time.sleep(0.36)

        successful = False
        while not successful:
            try:
                get_start = timeit.default_timer()
                games = leaguedashplayerbiostats.LeagueDashPlayerBioStats(season = f"{season}-{str(season+1)[-2:]}", timeout=round(2*sum(time_taken)/len(time_taken), 0))
                get_end = timeit.default_timer()
                time_taken.append(get_end-get_start)
                successful = True
            except Exception as e:
                if verbose:
                    print(f"Timeout error on season {season}-{str(season+1)[-2:]}, taking a 3 second break before resuming")
                time.sleep(3)

        gamelog = games.get_data_frames()[0]
        gamelog["SEASON"] = season
        lst.append(gamelog)
    season_infos = pd.concat(lst, axis = 0, ignore_index = True)
    if saveas:
        season_infos.to_csv(f"{saveas}.csv",index=False)
    return season_infos

# Step 4: Creating Dataset

### We're going to combine all the information we have now to create a dataset

In [146]:
gamelogs = pd.read_csv("../common-datasets/all_gamelogs.csv")
boxscores = pd.read_csv("../common-datasets/all_boxscores_concatenated.csv")
season_stats_per_poss = pd.read_csv("../common-datasets/player-seasons_per_possession.csv")
season_stats_per_game = pd.read_csv("../common-datasets/player-seasons_per_game.csv")
season_stats_totals = pd.read_csv("../common-datasets/player-seasons_totals.csv")
season_stats_advanced_totals = pd.read_csv("../common-datasets/player-seasons-advanced_totals.csv")
playerinfo = pd.read_csv("../common-datasets/commonplayerinfo.csv")

In [133]:
starters_boxscores = boxscores.dropna(subset = ["START_POSITION"])
starters_ids = list(starters_boxscores["PLAYER_ID"].unique())

In [77]:
player_info_dict = {k:dict() for k in starters_ids}
for player_id in player_info_dict.keys():
    #trim all the player-seasons we have so we're only looking at the relevant players
    per_poss = season_stats_per_poss[season_stats_per_poss["PLAYER_ID"] == player_id].reset_index()
    per_poss = per_poss.rename(index = {i:yr for i, yr in enumerate(list(per_poss["SEASON"]))})
    
    totals = season_stats_totals[season_stats_totals["PLAYER_ID"] == player_id].reset_index()
    totals = totals.rename(index = {i:yr for i, yr in enumerate(list(totals["SEASON"]))})
    
    advanced_totals = season_stats_advanced_totals[season_stats_totals["PLAYER_ID"] == player_id].reset_index()
    advanced_totals = advanced_totals.rename(index = {i:yr for i, yr in enumerate(list(advanced_totals["SEASON"]))})
    
    common_info =  playerinfo[playerinfo["PERSON_ID"] == player_id].reset_index()
    
    #make the entry a dictionary so we can store all the seasons
    player_info_dict[player_id] = dict()
        
    #put in the outer level of the dictionary all the stats that dont change with each season
    try:
        pick = int(common_info.at[0,"DRAFT_NUMBER"])
        if pick > 60:
            pick = "Undrafted"
    except:
        pick = "Undrafted"    
    player_info_dict[player_id]["seasons_available"] = set()
    player_info_dict[player_id]["first_season"] = int(common_info.at[0,"FROM_YEAR"])   
    player_info_dict[player_id]["pick"] = pick
    
    #now look at all teh different seasons
    for season in per_poss["SEASON"].unique():
        try:
            season = int(season)
            player_info_dict[player_id]["seasons_available"].add(season)
            player_info_dict[player_id][season] = dict()
            player_season = player_info_dict[player_id][season]

            #efficiency stats - total fgm, 3pm, fga
            player_season["FG3M"] = int(totals.at[season, "FG3M"])
            player_season["FGM"] = int(totals.at[season, "FGM"])
            player_season["FGA"] = int(totals.at[season, "FGA"])

            #turnover stats - turnovers per possession
            player_season["TOV"] = float(per_poss.at[season, "TOV"])

            #free throw stats - raw fta, fga
            player_season["FTM"] = int(totals.at[season, "FTM"])
            player_season["FGA"] = int(totals.at[season, "FGA"])

            #rebounding stats - season OREB% and DREB%
            player_season["OREB_PCT"] = float(advanced_totals.at[season, "OREB_PCT"])
            player_season["DREB_PCT"] = float(advanced_totals.at[season, "DREB_PCT"])
        except:
            display(totals)

In [78]:
draft_pick_dict = {k:dict() for k in range(1,61)}
draft_pick_dict["Undrafted"] = dict()
for draft_pick in draft_pick_dict.keys():
    draft_pick_dict[draft_pick]["num_pl"] = 0
    draft_pick_dict[draft_pick]["stats"] = dict()
    for pid, d in player_info_dict.items():
        if d["pick"] == draft_pick and d["first_season"] in d["seasons_available"] :
            draft_pick_dict[draft_pick]["num_pl"] += 1
            for k, v in d[d["first_season"]].items():
                try:
                    draft_pick_dict[draft_pick]["stats"][k] += v
                except:
                    draft_pick_dict[draft_pick]["stats"][k] = v
    for k, v in draft_pick_dict[draft_pick]["stats"].items():
        draft_pick_dict[draft_pick]["stats"][k] /= draft_pick_dict[draft_pick]["num_pl"]

In [149]:
"""this will generate a dictionary that maps game ids to a lot of information about the game. From game id, it maps to
home and away, which maps to all the different (relevant (relevant means played 3 minutes in a game at some point in their
career)) players, which then maps to the second they played in that game + their stats from the previous season
"""
gids = gamelogs[gamelogs["SEASON"] > 2002]["GAME_ID"].unique()
game_info_dict = {k:{"home":dict(), "away":dict()} for k in gids}
for game_id in gids:
    relevant_games = gamelogs[(gamelogs['GAME_ID'] == game_id)]
    relevant_boxscores = boxscores[(boxscores['GAME_ID'] == game_id)]
    relevant_games.reset_index(drop=True, inplace=True)
    current_season = relevant_games.loc[0, "SEASON"]
    
    # second step.... add the appropriate team id to home vs away
    if "@" in relevant_games.loc[0, "MATCHUP"]:
        away_team_row = 0
        home_team_row = 1      
    else:
        away_team_row = 1
        home_team_row = 0
        
    game_info_dict[game_id]["winner"] = relevant_games.loc[0, "TEAM_ID"] if relevant_games.loc[0, "WL"] == "W" else relevant_games.loc[1, "TEAM_ID"]
    game_info_dict[game_id]["away"]["team_id"] = relevant_games.loc[away_team_row, "TEAM_ID"]
    away_team_id = relevant_games.loc[away_team_row, "TEAM_ID"]
    game_info_dict[game_id]["home"]["team_id"] = relevant_games.loc[home_team_row, "TEAM_ID"]
    home_team_id = relevant_games.loc[home_team_row, "TEAM_ID"]
    
    #log the win percentages
    game_info_dict[game_id]["away"]["W_PCT"] = relevant_games.loc[away_team_row, "W_PCT"]
    game_info_dict[game_id]["home"]["W_PCT"] = relevant_games.loc[home_team_row, "W_PCT"]
    
    #start logging the players
    game_info_dict[game_id]["home"]["players"] = dict()
    game_info_dict[game_id]["away"]["players"] = dict()
    for row in relevant_boxscores.itertuples(): #we are essentially iterating through the players here 
        sec_played = str_to_mins(row.MIN) #seconds played that match
        start_pos = row.START_POSITION #psoition they started in (nan if not a starter)
        playername = row.PLAYER_NAME #name of the player
        pid = row.PLAYER_ID
        
        if start_pos in {"F","G","C"}: #starter in the game
            loc_tag = "home" if row.TEAM_ID == home_team_id else "away" #quick way t osee if we are home or away
            game_info_dict[game_id][loc_tag]["players"][playername] = dict() #ety dictionary to store stats
            game_info_dict[game_id][loc_tag]["players"][playername] = {"sec_played":sec_played, "start_pos":start_pos}
            try:
                game_info_dict[game_id][loc_tag]["players"][playername]["stats"] = player_info_dict[pid][current_season - 1]
            except KeyError:
                try:
                    game_info_dict[game_id][loc_tag]["players"][playername]["stats"] = draft_pick_dict[player_info_dict[pid]["pick"]]["stats"]
                except Exception as e:
                    print(playername, current_season, e)

In [150]:
base_columns = ["shooting", "turnovers", "OREB%", "DREB%", "free_throws", "W_PCT"]
real_columns = []
for colname in base_columns:
    real_columns.append("hm_" + colname)
    real_columns.append("aw_" + colname)
real_columns = sorted(real_columns)
real_columns.append("result")
unencoded_df = pd.DataFrame(columns = real_columns, index = range(len(game_info_dict.keys())))

In [151]:
unencoded_df

,aw_DREB%,aw_OREB%,aw_W_PCT,aw_free_throws,aw_shooting,aw_turnovers,hm_DREB%,hm_OREB%,hm_W_PCT,hm_free_throws,hm_shooting,hm_turnovers,result
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
bad_data = []
for index, (game_id, obj) in enumerate(game_info_dict.items()):
    winner_id = obj["winner"]
    home_dict = obj["home"]
    away_dict = obj["away"]
    unencoded_df.at[index, "result"] = 1 if winner_id == home_dict["team_id"] else 0
    
    for dic in [home_dict, away_dict]:
        loc_prefix = "hm_" if dic == home_dict else "aw_"
        fg = 0
        threep = 0
        fga = 0
        tov = 0
        oreb = 0
        dreb = 0
        ftm = 0   
        for player_name, player_obj in dic["players"].items():
            try: #this is an individual player loop.... if a single player can't be accessed, its ok to skip him
                fg += player_obj["stats"]["FGM"]
                threep += player_obj["stats"]["FG3M"]
                fga += player_obj["stats"]["FGA"]
                tov += player_obj["stats"]["TOV"]
                oreb += player_obj["stats"]["OREB_PCT"]
                dreb += player_obj["stats"]["DREB_PCT"]
                ftm += player_obj["stats"]["FTM"]
            except Exception as e:
                print(player_name, player_obj, e)
        try: #this is at a team level... if we fail here, we should not have this trianing example
            unencoded_df.at[index, loc_prefix + "shooting"] = (fg+0.5*threep)/fga
            unencoded_df.at[index, loc_prefix + "turnovers"] = tov
            unencoded_df.at[index, loc_prefix + "free_throws"] = ftm/fga
            unencoded_df.at[index, loc_prefix + "OREB%"] = oreb/5
            unencoded_df.at[index, loc_prefix + "DREB%"] = dreb/5
            unencoded_df.at[index, loc_prefix + "W_PCT"] = dic["W_PCT"]
        except Exception as e:
                print(game_id, e)
unencoded_df = unencoded_df.drop(unencoded_df.index[bad_data])

21201214 float division by zero
21201214 float division by zero


In [75]:
game_info_dict[21201214]

{'home': {'team_id': 1610612738, 'players': {}},
 'away': {'team_id': 1610612754, 'players': {}},
 'winner': 1610612754}

In [153]:
unencoded_df

,aw_DREB%,aw_OREB%,aw_W_PCT,aw_free_throws,aw_shooting,aw_turnovers,hm_DREB%,hm_OREB%,hm_W_PCT,hm_free_throws,hm_shooting,hm_turnovers,result
0,0.1422,0.052,0.683,0.29271,0.490946,0.171,0.1286,0.0566,0.5,0.217406,0.464822,0.156,0
1,0.11466,0.05309,0.444,0.182224,0.466284,0.1485,0.1422,0.052,0.679,0.29271,0.490946,0.171,1
2,0.1422,0.052,0.675,0.29271,0.490946,0.171,0.1248,0.0394,0.688,0.211692,0.503882,0.152,1
3,0.135,0.0458,0.633,0.222043,0.47576,0.152,0.141,0.0468,0.684,0.280756,0.495776,0.179,1
4,0.1286,0.0566,0.519,0.217406,0.464822,0.156,0.141,0.0468,0.679,0.280756,0.495776,0.179,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19394,0.118664,0.0258182,0.217,0.181968,0.47971,0.132318,0.1476,0.0498,0.632,0.21329,0.519287,0.139,1
19395,0.124,0.0472,0.5,0.237133,0.506975,0.106,0.118664,0.0258182,0.238,0.181968,0.47971,0.132318,0
19396,0.155,0.0472,1,0.213764,0.50632,0.152,0.133464,0.0374182,0,0.170639,0.485494,0.142318,0
19397,0.159255,0.0519636,0.483,0.212027,0.513316,0.158273,0.118,0.0536,0.54,0.249592,0.521775,0.112,1


In [154]:
unencoded_df.to_csv("training_dataset_ff_plus_naive.csv", index = False)